In [6]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.7 MB 5.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/8.7 MB 7.0 MB/s eta 0:00:01
   ------------------ --------------------- 3.9/8.7 MB 7.3 MB/s eta 0:00:01
   ------------------------- -------------- 5.5/8.7 MB 7.3 MB/s eta 0:00:01
   ------------------------------- -------- 6.8/8.7 MB 7.1 MB/s eta 0:00:01
   ------------------------------------- -- 8.1/8.7 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------  8.7/8.7 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.4 MB ? eta -:--:--
    --------------------------------------- 0.8/38.4 MB 4.1 MB/s eta 0:00:10
   -- ------------------------------------- 2.4/38.4 MB 5.8 MB/s eta 0:00:07
   --- ------------------------------------ 3.7/38.4 MB 6.1 MB/s eta 0:00:06
   ----- -------------

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

In [9]:
# Load data
df = pd.read_csv("F:/Python/Ostad Python/Jupyter notebook/synthetic_ecommerce_data.csv")
df

,Transaction_ID,Customer_ID,Product_ID,Transaction_Date,Units_Sold,Discount_Applied,Revenue,Clicks,Impressions,Conversion_Rate,Category,Region,Ad_CTR,Ad_CPC,Ad_Spend
0,8b460852-7c64-46fa-998b-b0976879d082,Customer_65,Product_224,2024-10-06,134,0.14,305.54,11,65,0.17,Electronics,Europe,0.0180,0.55,9.90
1,418612e7-8744-4ba3-bb0c-105b47e2a968,Customer_1910,Product_584,2024-10-29,109,0.30,1102.19,15,201,0.07,Home Appliances,Asia,0.1589,0.40,63.56
2,5bc3b98f-cb0c-4b12-947c-df8bbb35a73e,Customer_2306,Product_374,2024-04-04,116,0.04,471.29,16,199,0.08,Toys,Asia,0.0596,1.50,89.40
3,28fb67c8-e8c0-447a-841c-f760730de0eb,Customer_17206,Product_220,2024-08-25,125,0.20,980.26,12,355,0.03,Clothing,Europe,0.0444,0.44,19.54
4,8bee087a-a8a9-45bb-89d7-04d1710f1b00,Customer_16033,Product_358,2024-05-05,132,0.07,803.76,44,355,0.12,Books,North America,0.1270,0.53,67.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,90320769-c92d-4142-b112-4bac46d44007,Customer_2613,Product_472,2024-04-21,179,0.11,255.11,41,428,0.10,Electronics,Europe,0.1073,1.48,158.80
99996,e91a3cfa-bd5b-475a-825d-6efe939ff974,Customer_19127,Product_787,2024-09-17,178,0.12,1302.60,6,363,0.02,Books,North America,0.1164,0.36,41.90
99997,b0e10e62-e7c4-4092-876a-12abc9ebe3e8,Customer_4685,Product_745,2024-10-23,212,0.26,491.36,5,419,0.01,Electronics,North America,0.1235,0.20,24.70
99998,97980811-e107-4369-888c-31cd8763471f,Customer_16377,Product_39,2024-01-03,180,0.09,598.91,41,12,3.42,Electronics,Europe,0.1566,0.96,150.34


In [10]:
# Data Preprocessing
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
df['Year'] = df['Transaction_Date'].dt.year
df['Month'] = df['Transaction_Date'].dt.month
df['Day'] = df['Transaction_Date'].dt.day
df['Week'] = df['Transaction_Date'].dt.isocalendar().week
df['Revenue_per_Unit'] = df['Revenue'] / df['Units_Sold']
df['Ad_Effectiveness'] = df['Ad_CTR'] / df['Ad_CPC']

In [11]:
# Handle categorical variables using LabelEncoder
label_encoder = LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category'])
df['Region'] = label_encoder.fit_transform(df['Region'])

In [12]:
# Select features and target variable
X = df[['Units_Sold', 'Discount_Applied', 'Clicks', 'Impressions', 
        'Conversion_Rate', 'Category', 'Region', 'Ad_CTR', 'Ad_CPC', 'Ad_Spend', 
        'Year', 'Month', 'Day','Week','Revenue_per_Unit','Ad_Effectiveness']]
Y = df['Revenue']

# Split dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [13]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Step 1: Train a Linear Regression model
model = LinearRegression()

# Step 2: Fit the model on the scaled training data
model.fit(X_train_scaled, Y_train)

# Step 3: Make predictions on the scaled test set
Y_pred = model.predict(X_test_scaled)

# Step 4: Evaluate the model
mse = mean_squared_error(Y_test, Y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R² Score: {r2}')

Mean Absolute Error: 123.54132847875147
Mean Squared Error: 30273.782265856153
Root Mean Squared Error: 173.99362708402901
R² Score: 0.8341803658527637


In [16]:
# Display predicted vs actual revenue
predicted_vs_actual = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
print(predicted_vs_actual.head(15))

       Actual    Predicted
75721  191.35   144.946456
80184  325.60   313.632416
19864  607.18   539.199588
76699  378.87   323.152422
92991  595.24   526.494194
76434  511.19   467.798521
84004   21.94   118.252842
80917  243.00   397.123417
60767  115.89    36.689379
50074  802.79  1031.674972
27701  867.96   861.701265
42141  139.26   136.905532
45080  157.86   330.633691
16638  196.44   158.718035
20424   30.19     4.504079


In [9]:
pd.set_option('display.max_columns', None)  # This will display all columns
print(df)

                             Transaction_ID     Customer_ID   Product_ID  \
0      8b460852-7c64-46fa-998b-b0976879d082     Customer_65  Product_224   
1      418612e7-8744-4ba3-bb0c-105b47e2a968   Customer_1910  Product_584   
2      5bc3b98f-cb0c-4b12-947c-df8bbb35a73e   Customer_2306  Product_374   
3      28fb67c8-e8c0-447a-841c-f760730de0eb  Customer_17206  Product_220   
4      8bee087a-a8a9-45bb-89d7-04d1710f1b00  Customer_16033  Product_358   
...                                     ...             ...          ...   
99995  90320769-c92d-4142-b112-4bac46d44007   Customer_2613  Product_472   
99996  e91a3cfa-bd5b-475a-825d-6efe939ff974  Customer_19127  Product_787   
99997  b0e10e62-e7c4-4092-876a-12abc9ebe3e8   Customer_4685  Product_745   
99998  97980811-e107-4369-888c-31cd8763471f  Customer_16377   Product_39   
99999  52f15362-a573-4060-8f70-462f82adda92  Customer_14281  Product_223   

      Transaction_Date  Units_Sold  Discount_Applied  Revenue  Clicks  \
0           20

In [10]:
#df.Category.value_counts()
print(df['Category'].unique())

[2 3 4 1 0]


In [11]:
# Hyperparameter tuning using RandomizedSearchCV for Linear Regression (though limited)
param_distributions = {
    'fit_intercept': [True, False]  # Only 'fit_intercept' is available for tuning in Linear Regression
}

random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_distributions, 
                                   n_iter=20, 
                                   cv=5, 
                                   random_state=42, 
                                   n_jobs=-1)

# Step 5: Fit the random search on the scaled training data
random_search.fit(X_train_scaled, Y_train)

# Step 6: Output the best parameters found
print(f'Best Parameters: {random_search.best_params_}')

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 2 is smaller than n_iter=20. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters: {'fit_intercept': True}


In [40]:
# Step 7: Make predictions using the best model from random search
Y_pred_best = random_search.best_estimator_.predict(X_test_scaled)

# Evaluate the tuned model
mse_best = mean_squared_error(Y_test, Y_pred_best)
rmse_best = mse_best ** 0.5
mae_best = mean_absolute_error(Y_test, Y_pred_best)
r2_best = r2_score(Y_test, Y_pred_best)

print(f'Best Model - Mean Absolute Error: {mae_best}')
print(f'Best Model - Mean Squared Error: {mse_best}')
print(f'Best Model - Root Mean Squared Error: {rmse_best}')
print(f'Best Model - R² Score: {r2_best}')

# Display predicted vs actual revenue for best model
predicted_vs_actual_best = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred_best})
print(predicted_vs_actual_best.head())

Best Model - Mean Absolute Error: 123.541328478753
Best Model - Mean Squared Error: 30273.782265856116
Best Model - Root Mean Squared Error: 173.9936270840289
Best Model - R² Score: 0.8341803658527639
       Actual   Predicted
75721  191.35  144.946456
80184  325.60  313.632416
19864  607.18  539.199588
76699  378.87  323.152422
92991  595.24  526.494194


### revenue prediction with other indicators

In [14]:
# New data for prediction
new_data = {
    'Units_Sold': [134, 50, 30],
    'Discount_Applied': [0.14, 0.1, 0.08],
    'Clicks': [11, 500, 300],
    'Impressions': [65, 5000, 3000],
    'Conversion_Rate': [0.17, 0.15, 0.12],
    'Category': [2, 2, 3],  # Assuming these are encoded values for 'Category'
    'Region': [1, 1, 2],     # Assuming these are encoded values for 'Region'
    'Ad_CTR': [0.0180, 0.05, 0.03],
    'Ad_CPC': [0.55, 0.5, 0.4],
    'Ad_Spend': [9.90, 1500, 900],
    'Year': [2024, 2023, 2023],
    'Month': [10, 1, 1],
    'Day': [6, 16, 17],
    'Week': [40, 2, 2],
    'Revenue_per_Unit': [2.280149, 22.04, 19.96],
    'Ad_Effectiveness': [0.032727, 0.5, 0.45]
}

# Convert the new data to a DataFrame
new_data_df = pd.DataFrame(new_data)

# Ensure to scale the new data using the same scaler used for training data
new_data_scaled = scaler.transform(new_data_df)  # Use the same scaler instance

# Use the trained model (best estimator from RandomizedSearchCV)
Y_new_pred = random_search.best_estimator_.predict(new_data_scaled)

# Output the predicted revenue
print(f'Predicted Revenue for new data: {Y_new_pred}')

Predicted Revenue for new data: [ 317.93526495 1948.17942169 1629.00616975]


### optional step for revenue prediction from actual prediction without other indicators

In [16]:
# Step 10: Evaluate the predictions using Linear Regression (optional step for comparison)
data = {'Actual': Y_test, 'Predicted': Y_pred}
df_lr = pd.DataFrame(data)

# Create and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(df_lr[['Predicted']], df_lr['Actual'])

# Make predictions with the Linear Regression model using new predicted values
new_predicted_values = [305.54, 1102.19, 598.91]  # Replace with your new predicted values
lr_predictions = lr_model.predict(pd.DataFrame({'Predicted': new_predicted_values}))

# Evaluate the Linear Regression model
lr_mse = mean_squared_error(df_lr['Actual'], lr_model.predict(df_lr[['Predicted']]))
print(f'Linear Regression Mean Squared Error: {lr_mse}')
print('Linear Regression Predictions:', lr_predictions)

Linear Regression Mean Squared Error: 30262.198655970813
Linear Regression Predictions: [ 304.1227396  1107.6294286   600.01774663]
